# *``lca_graphic``* : **A toolbox for daily LCA users**

---

**Purpose = to give understandable graphs for the daily LCA users to quickly analyze his LCA**
<br>
---
Author : Teo Lavisse, teo.lavisse@cea.fr       
---


## Initializing

In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
import json
import os
import matrix_utils as mu
import bw_processing as bp
import math
import warnings
import ipywidgets as widgets
from ipywidgets import interact
import bw2analyzer as ba
import seaborn as sns
import textwrap

from utils import *
from dashboards import *

In [2]:
bd.projects.set_current("Visualization")

### Importing Databases

In [3]:
bd.databases

Databases dictionary with 2 object(s):
	US EEIO 1.1
	biosphere3

In [4]:
datadir = Path("data")
datadir.mkdir(exist_ok=True)

In [5]:
bi.useeio11()

US EEIO 1.1 already present


In [6]:
df = bd.Database("US EEIO 1.1").nodes_to_dataframe()
df

,CAS number,categories,classifications,code,database,description,dqEntry,dqSystem,exchangeDqSystem,filename,id,location,modified,name,processDocumentation,type,unit,version
1150,,"(air, low population density)",NaN,6ca23b5d-83dc-3b02-bf39-8eabf9d41151,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1640,NaN,NaN,"(2,4-DICHLOROPHENOXY)ACETIC ACID COMPD. WITH 2...",NaN,emission,,NaN
2112,,"(soil, groundwater)",NaN,5b98f875-8d1c-3549-a7df-28d7d90e7ccb,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1427,NaN,NaN,"(2,4-DICHLOROPHENOXY)ACETIC ACID COMPD. WITH 2...",NaN,emission,,NaN
2341,,"(water, unspecified)",NaN,2ee4697d-b7f4-362b-86a4-94b644699500,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1134,NaN,NaN,"(2,4-DICHLOROPHENOXY)ACETIC ACID COMPD. WITH 2...",NaN,emission,,NaN
482,,"(water, unspecified)",NaN,93086e32-c013-3e34-a074-4760c72fe775,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1886,NaN,NaN,(4-CHLORO-2-METHYLPHENOXY)ACETIC ACID COMPD. W...,NaN,emission,,NaN
1185,,"(soil, groundwater)",NaN,b8889098-9a89-35c6-b226-07cb66c83217,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,2160,NaN,NaN,(4-CHLORO-2-METHYLPHENOXY)ACETIC ACID COMPD. W...,NaN,emission,,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2244,,"(air, unspecified)",NaN,d9a5b786-d06c-44af-a088-b070aa605d9b,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,2370,NaN,NaN,trifluralin,NaN,emission,,NaN
350,7440622.0,"(water, unspecified)",NaN,63e8256e-8549-11e0-9d78-0800200c9a66,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1577,NaN,NaN,vanadium,NaN,emission,,NaN
1337,7440622.0,"(air, unspecified)",NaN,591b0a62-8064-4697-86ed-47bfa1f8b5e6,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,1407,NaN,NaN,vanadium,NaN,emission,,NaN
1554,7440622.0,"(soil, industrial)",NaN,1a5850a0-0069-4b73-bb91-7a61e8d45ae5,US EEIO 1.1,NaN,NaN,NaN,NaN,NaN,971,NaN,NaN,vanadium,NaN,emission,,NaN


In [7]:
def safe_joiner(x):
    try:
        if not x or np.isnan(x):
            return None
    except TypeError:
        pass
    return "::".join(x)

In [8]:
df['categories'] = df['categories'].apply(safe_joiner)


In [9]:
def safe_converter(x):
    try:
        return str(int(x))
    except:
        return None

In [10]:
df['CAS number'] = df['CAS number'].apply(safe_converter)


In [11]:
df.to_csv(datadir / "useeio_nodes.csv", index=False)

In [12]:

bd.Database("US EEIO 1.1").edges_to_dataframe().to_csv(datadir / "useeio_edges.csv", index=False)

Getting activity data


100%|███████████████████████████████████████████████████████████████████████████| 2649/2649 [00:00<00:00, 65428.33it/s]


Adding exchange data to activities


100%|███████████████████████████████████████████████████████████████████████| 162926/162926 [00:10<00:00, 15582.95it/s]


Filling out exchange data


100%|███████████████████████████████████████████████████████████████████████████| 2649/2649 [00:00<00:00, 78188.83it/s]


Creating DataFrame
Compressing DataFrame


In [13]:
product = next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'product')


In [14]:
bi.bw2setup() #Importing elementary flows, LCIA methods and some other data

Biosphere database already present!!! No setup is needed


In [15]:
bd.databases


Databases dictionary with 2 object(s):
	US EEIO 1.1
	biosphere3

In [16]:
df=bd.Database("US EEIO 1.1")


###  Get activities and methods

Get 2 activities and methods from the database

In [5]:
act3=next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'product')
act3

'Irradiation apparatuses; at manufacturer' (, United States, ('31-33: Manufacturing', '3345: Navigational, Measuring, Electromedical, and Control Instruments Manufacturing'))

In [6]:
window_metal=next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'product' if node['name']=="Metal windows, doors, and architectural products; at manufacturer")
window_metal

'Metal windows, doors, and architectural products; at manufacturer' (, United States, ('31-33: Manufacturing', '3323: Architectural and Structural Metals Manufacturing'))

In [7]:
window_wood=next(node for node in bd.Database("US EEIO 1.1") if node['type'] == 'product' if node['name']=="Wooden windows, door, and flooring; at manufacturer")
window_wood

'Wooden windows, door, and flooring; at manufacturer' (, United States, ('31-33: Manufacturing', '3219: Other Wood Product Manufacturing'))

In [8]:
fu={window_metal:1,window_wood:1}

In [9]:
methods = [
    ('Impact Potential', 'GCC'),
    ('Impact Potential', 'HTOX'),
    ('Resource Use', 'LAND'),
    ('Resource Use', 'WATR'),
]

### Defining colors

In [10]:
#define standard color palette:
colors = ["#F08C2E", "#7f6000", "#72AF42", "#A32683"]

#create longer color list for complex figures
color_div_YlBr=sns.color_palette('YlOrBr',6)
color_seq_green=sns.color_palette('Greens',6)
color_seq_RdPu=sns.color_palette('RdPu',6)
color_seq_org=sns.color_palette('Oranges',5)

colors.extend(colors)
# colors.extend(color_div_YlBr)
# colors.extend(color_seq_green)
# colors.extend(color_seq_RdPu)
# colors.extend(color_seq_org)

df_color=pd.DataFrame(index=methods,data=[colors[c] for c in range(len(methods))]).T

##  Perform basic lca computations

In [11]:
df=lca_comparison(fu, [methods[0]])
df

,"(Impact Potential, GCC)"
"Metal windows, doors, and architectural products; at manufacturer",0.567139
"Wooden windows, door, and flooring; at manufacturer",0.384134


In [12]:
df_norm=df.T.apply(lambda x: x/x.max(), axis=1)*100 #to normalize the results for each impact category
df_norm

,"Metal windows, doors, and architectural products; at manufacturer","Wooden windows, door, and flooring; at manufacturer"
"(Impact Potential, GCC)",100.0,67.731889


## Perform a basic contribution analysis

In [13]:
act_topscore(fu)

'Metal windows, doors, and architectural products; at manufacturer' (, United States, ('31-33: Manufacturing', '3323: Architectural and Structural Metals Manufacturing'))

In [12]:
df=contributions_df(window_metal,methods[0],limit=5,limit_type='number',norm=True)
df

,"(Impact Potential, GCC)"
Natural gas; at consumer,4.746876
Truck transport,6.116044
Coal; at mine,6.409533
Electricity; at consumer,21.308793
"Primary iron, steel, and ferroalloy products; at manufacturer",27.927123
Others,33.491631


In [13]:
df.values.sum()

100.0

## Display all thes informations in a nice dashboard

### Dashboard compare

In [14]:
compare(fu, methods, sharex=False, cols=2, func_unit="kWh")

### Dashboard hotspots

In [15]:
hotspots(fu, methods, limit=0.02)

### Dashboard impact_transfer

In [19]:
impact_transfer({window_metal:1,window_wood:1,act3:1}, methods, reference_category=('Impact Potential', 'GCC'),limit=5, cols=2, func_unit="kWh")

## Generic function : lca_graphic

###### lca_graphic(fu,methods, reference_category=('Impact Potential', 'GCC'), func_unit="kg")

In [20]:
lca_graphic(fu,methods)